In [69]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

## Problem 3

In [452]:
df = pd.read_csv("fairness_data.csv")
df.dtypes.unique()
dfq = df.astype(int)

### (b) Train a fairness through unawareness classiﬁer $\hat{f}_{FTU}$ that predicts Y using all the non-sensitive features. Use a gradient boosting classiﬁer.

In [453]:
Unawareness_D = dfq.copy()
Z = Unawareness_D.pop('Z')
Y = Unawareness_D.pop('income')

1. splite data into training and testing, and normalize

In [454]:
X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(Unawareness_D, Y, Z, test_size=0.3, random_state=13)


In [455]:
gb_classifier = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=4,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=0.8,
    max_features='sqrt',
    random_state=42)
gb_classifier.fit(X_train, Y_train)
Y_pred = gb_classifier.predict(X_test)
print(f"Accuracy: {accuracy_score(Y_test, Y_pred):.4f}")


Accuracy: 0.8466


2. Training model and report the accuracy on testing data

In [456]:
gb_classifier = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=4,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=0.8,
    max_features='sqrt',
    random_state=42)
gb_classifier.fit(X_train, Y_train)

GradientBoostingClassifier(learning_rate=0.05, max_depth=4, max_features='sqrt',
                           n_estimators=200, random_state=42, subsample=0.8)

In [457]:
Y_pred = gb_classifier.predict(X_test)
print(f"Accuracy: {accuracy_score(Y_test, Y_pred):.4f}")


Accuracy: 0.8466


3. check fairness measurement

In [458]:

predictions = gb_classifier.predict_proba(X_test)[:,1]


group1 = (Z_test==1)
group0 = (Z_test==0)
DP = predictions[group1].mean() - predictions[group0].mean()

group1_Y_1 = ((Z_test==1) & (Y_test==1))
group0_Y_1 = ((Z_test==0) & (Y_test==1))
EO = predictions[group1_Y_1].mean() - predictions[group0_Y_1].mean()

PP = mean_absolute_error(Y_test[group1], predictions[group1]) - mean_absolute_error(Y_test[group0], predictions[group0])

print(f'DP= {DP:.4f}, EO= {EO:.4f}, PP= {PP:.4f}')

DP= -0.1673, EO= -0.1816, PP= -0.2399


### (c) Train a classiﬁer $\hat{f}(x, z)$ using all the features, including the sensitive attribute Z. And then deﬁne a new classiﬁer by marginalizing out the sensitive attribute

1. train classifier using all features

In [459]:
Awareness_D = dfq.copy()
Z = Awareness_D['Z']
Y = Awareness_D.pop('income')

In [ ]:
X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(Awareness_D, Y, Z, test_size=0.3, random_state=13)


In [467]:
gb_classifier = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=0.8,
    max_features='sqrt',
    random_state=42)
gb_classifier.fit(X_train, Y_train)


GradientBoostingClassifier(learning_rate=0.05, max_features='sqrt',
                           n_estimators=200, random_state=42, subsample=0.8)

2. Deﬁne a new classiﬁer by marginalizing out the sensitive attribute and test on tesing data

In [468]:
Z1 = (Z==1).sum()/len(Z)  #mariginal pro

X_test_1 = X_test.copy()
X_test_1['Z'] = 1
#X_test1_scaled = scaler.transform(X_test_1)
X_test_0 = X_test.copy()
X_test_0['Z'] = 0
#X_test0_scaled = scaler.transform(X_test_0)


Y_pred_prob = (gb_classifier.predict_proba(X_test_1) + gb_classifier.predict_proba(X_test_0))/2
Y_pred = pd.DataFrame(Y_pred_prob).idxmax(axis=1)


print(f"Accuracy: {accuracy_score(Y_test, Y_pred):.4f}")

Accuracy: 0.8408


3. check fairness measurement

In [469]:
#predictions = gb_classifier.predict(scaler.transform(Unawareness_D))
predictions = Y_pred_prob[:,1]

# Split data by the sensitive attribute 'Z' and check the accuracy in each group
group1 = (Z_test==1)
group0 = (Z_test==0)
DP = predictions[group1].mean() - predictions[group0].mean()

group1_Y_1 = ((Z_test==1) & (Y_test==1))
group0_Y_1 = ((Z_test==0) & (Y_test==1))
EO = predictions[group1_Y_1].mean() - predictions[group0_Y_1].mean()

PP = mean_absolute_error(Y_test[group1], predictions[group1]) - mean_absolute_error(Y_test[group0], predictions[group0])

print(f'DP= {DP:.4f}, EO= {EO:.4f}, PP= {PP:.4f}')

DP= -0.0525, EO= -0.0173, PP= -0.1875


### (e)     Next we will just use the real data, dropping the synthetic sensitive attribute Z and instead treating sex as the sensitive attribute. Run the same analysis as in parts (b) and (c) using the real sensitive attribute. How do the results compare to the synthetic sensitive attribute?

1. fairness through unawareness

In [470]:
Unawareness_D = dfq.copy()
Unawareness_D = Unawareness_D.drop(columns=["Z"])
Z = Unawareness_D.pop('sex')
Y = Unawareness_D.pop('income')

X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(Unawareness_D, Y, Z, test_size=0.3, random_state=13)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [471]:
Z.value_counts()

sex
0    30527
1    14695
Name: count, dtype: int64

In [475]:
gb_classifier = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=4,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=0.8,
    max_features='sqrt',
    random_state=42)
gb_classifier.fit(X_train_scaled, Y_train)
Y_pred = gb_classifier.predict(X_test_scaled)
print(f"Accuracy: {accuracy_score(Y_test, Y_pred):.4f}")

Accuracy: 0.8462


In [476]:
predictions = gb_classifier.predict_proba(X_test_scaled)[:,1]

group1 = (Z_test==1)
group0 = (Z_test==0)
DP = predictions[group1].mean() - predictions[group0].mean()

group1_Y_1 = ((Z_test==1) & (Y_test==1))
group0_Y_1 = ((Z_test==0) & (Y_test==1))
EO = predictions[group1_Y_1].mean() - predictions[group0_Y_1].mean()

PP = mean_absolute_error(Y_test[group1], predictions[group1]) - mean_absolute_error(Y_test[group0], predictions[group0])

print(f'DP= {DP:.4f}, EO= {EO:.4f}, PP= {PP:.4f}')

DP= -0.0585, EO= -0.0781, PP= -0.0664


In [477]:
gb_classifier.predict_proba(X_test_scaled)

array([[0.71109887, 0.28890113],
       [0.86679671, 0.13320329],
       [0.98059361, 0.01940639],
       ...,
       [0.86568779, 0.13431221],
       [0.91881065, 0.08118935],
       [0.88711503, 0.11288497]])

2. fairness of $\hat{f}^{inv}$

In [478]:
Awareness_D = dfq.copy()
Awareness_D = Awareness_D.drop(columns=['Z'])
Z = Awareness_D['sex']
Y = Awareness_D.pop('income')

X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(Awareness_D, Y, Z, test_size=0.3, random_state=13)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [481]:
gb_classifier1 = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=0.8,
    max_features='sqrt',
    random_state=42)
gb_classifier1.fit(X_train_scaled, Y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features='sqrt',
                           n_estimators=200, random_state=42, subsample=0.8)

In [482]:
Z1 = (Z==1).sum()/len(Z)  #mariginal pro

X_test_1 = X_test.copy()
X_test_1['sex'] = 1
X_test1_scaled = scaler.transform(X_test_1)
X_test_0 = X_test.copy()
X_test_0['sex'] = 0
X_test0_scaled = scaler.transform(X_test_0)


Y_pred_prob = (gb_classifier1.predict_proba(X_test1_scaled) + gb_classifier1.predict_proba(X_test0_scaled))/2
Y_pred = pd.DataFrame(Y_pred_prob).idxmax(axis=1)


print(f"Accuracy: {accuracy_score(Y_test, Y_pred):.4f}")

Accuracy: 0.8461


In [483]:
Y_pred_prob

array([[0.71916752, 0.28083248],
       [0.865431  , 0.134569  ],
       [0.97226355, 0.02773645],
       ...,
       [0.868044  , 0.131956  ],
       [0.92213622, 0.07786378],
       [0.86652548, 0.13347452]])

In [484]:
predictions = Y_pred_prob[:,1]

# Split data by the sensitive attribute 'Z' and check the accuracy in each group
group1 = (Z_test==1)
group0 = (Z_test==0)
DP = predictions[group1].mean() - predictions[group0].mean()

group1_Y_1 = ((Z_test==1) & (Y_test==1))
group0_Y_1 = ((Z_test==0) & (Y_test==1))
EO = predictions[group1_Y_1].mean() - predictions[group0_Y_1].mean()

PP = mean_absolute_error(Y_test[group1], predictions[group1]) - mean_absolute_error(Y_test[group0], predictions[group0])

print(f'DP= {DP:.4f}, EO= {EO:.4f}, PP= {PP:.4f}')

DP= -0.0539, EO= -0.0670, PP= -0.0655
